<a href="https://colab.research.google.com/github/dataitgirls4/team_3/blob/miree/%EC%A4%80%EB%8B%A8%EC%96%B4_%ED%86%A0%ED%81%B0%ED%99%94%EB%A5%BC_%ED%86%B5%ED%95%9C_%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%A0%95%EA%B7%9C%ED%99%94%EC%99%80_Early_Stopping_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 열기

네이버 영화평 데이터를 연다.

In [1]:
import pandas as pd

# 파일 열기

In [2]:
nsmc = pd.read_csv('https://github.com/e9t/nsmc/raw/master/ratings_train.txt', sep='\t')

데이터의 초반 5개를 확인한다.

In [3]:
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


데이터의 크기를 확인한다.

In [4]:
nsmc.shape

(150000, 3)

여기서는 2000개만으로 감성분석 실습을 진행한다.

In [5]:
nsmc = nsmc.loc[:1999]

# 준단어 토큰화

인터넷 글들은 맞춤법이나 띄어쓰기를 잘 지키지 않으므로 형태소 분석 대신 `sentencepiece`를 이용해서 준단어 토큰화를 한다.
영화평을 `nsmc.txt` 파일로 저장합니다.

In [6]:
with open('nsmc.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(nsmc['document']))

준단어 토큰을 학습한다. 토큰의 종류는 총 3000개로 한다.

In [7]:
pip install sentencepiece

     |████████████████████████████████| 1.1MB 6.9MB/s 


In [8]:
from sentencepiece import SentencePieceTrainer
SentencePieceTrainer.Train('--input=nsmc.txt --model_prefix=nsmc --vocab_size=3000')

In [9]:
from sentencepiece import SentencePieceProcessor
sp = SentencePieceProcessor()
sp.Load("nsmc.model")

True

In [10]:
sp.encode_as_pieces(nsmc.loc[0, 'document'])

['▁아', '▁더빙', '..', '▁진짜', '▁짜증나', '네요', '▁목소리']

# 단어 문서 행렬(TDM) 만들기

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

옵션 가운데 `lowercase=False`로 설정하면 영어의 대문자를 소문자로 변환을 하지 않는다.


In [12]:
cv = CountVectorizer(lowercase=False, tokenizer=sp.encode_as_pieces)

In [13]:
tdm = cv.fit_transform(nsmc['document'])

# 훈련용 데이터와 테스트용 데이터 나누기

새로운 데이터가 얼마나 일반화되어 있는지 측정하기 위해 훈련용과 테스트용 데이터를 나눈다.
- `x_train` : 훈련용 데이터 (리뷰)
- `y_train` : 훈련용 데이터 (0, 1로 분류된 감정)
- `x_test` : 테스트용 데이터 (리뷰)
- `y_test` : 테스트용 데이터 (0, 1로 분류된 감정)
`sklearn.model_selection`의 `train_test_split` 모듈을 사용한다.



In [14]:
from sklearn.model_selection import train_test_split

In [34]:
x = tdm
y = nsmc['label']

전체 데이터의 20%를 테스트용 데이터로 분할한다.

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 저장

전처리 결과를 저장한다.

In [17]:
import joblib

`cv`에서 `tokenizer`로 사용하고 있는 `sp.encode_as_pieces`는 C++ 언어로 작성된 함수를 Python에서 불러와 사용하는 것으로 `joblib`으로 저장할 수 없다. 따라서 여기서는 빈 값(`None`)으로 바꾸어놓는다.

In [18]:
cv.tokenizer = None

`joblib`으로 저장한다.

In [19]:
joblib.dump((cv, x_train, x_test, y_train, y_test), 'nsmc.pkl')

['nsmc.pkl']

# 데이터 불러오기

In [20]:
import joblib

cv, x_train, x_test, y_train, y_test = joblib.load('nsmc.pkl')

In [21]:
x_train.shape

(1600, 3019)

# 모형

In [22]:
import tensorflow as tf

`kernel_regularizer`에는 정규화를 위한 페널티 항을 추가한다. `l1`, `l2` 그리고 `l1_l2`를 사용할 수 있다.

In [41]:
model = tf.keras.models.Sequential()
model.add(
    tf.keras.layers.Dense(
        1, 
        input_shape=(3019,), 
        activation='sigmoid',
        kernel_regularizer=tf.keras.regularizers.l2(0.001)))

In [46]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 3020      
Total params: 3,020
Trainable params: 3,020
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 학습

In [48]:
x_train.toarray().shape

(1600, 3019)

In [49]:
y_train.values.shape

(1600,)

In [50]:
callbacks

0

In [51]:
model.fit(x_train.toarray(), y_train.values, epochs=100, validation_split=0.1,
          callbacks=[tf.keras.callbacks.EarlyStopping()])

Epoch 1/100
45/45 [==============================] - 0s 4ms/step - loss: 0.4350 - accuracy: 0.9361 - val_loss: 0.5800 - val_accuracy: 0.7312
Epoch 2/100
45/45 [==============================] - 0s 2ms/step - loss: 0.4302 - accuracy: 0.9382 - val_loss: 0.5800 - val_accuracy: 0.7375
Epoch 3/100
45/45 [==============================] - 0s 2ms/step - loss: 0.4273 - accuracy: 0.9396 - val_loss: 0.5803 - val_accuracy: 0.7312


In [52]:
model.evaluate(x_test.toarray(), y_test.values, verbose=0)

[0.63648521900177, 0.7124999761581421]

# 모형 평가

In [53]:
model.evaluate(x_test.toarray(), y_test.values, verbose=0)

[0.63648521900177, 0.7124999761581421]

# 계수 확인

In [55]:
weights, _ = model.trainable_weights

In [56]:
import pandas as pd
token_weight = pd.DataFrame({'토큰': cv.get_feature_names(), '가중치':weights.numpy().flat})

부정단어를 확인한다.

In [57]:
token_weight.sort_values('가중치').head()

,토큰,가중치
1568,냐,-0.514057
774,▁쓰레기,-0.457692
779,▁아까운,-0.434306
1204,▁최악의,-0.425058
443,▁돈,-0.414684


긍정단어를 확인한다.

In [58]:
token_weight.sort_values('가중치').tail()

,토큰,가중치
1094,▁정말,0.433496
1,!!,0.475573
536,▁명작,0.504365
1202,▁최고의,0.509873
1201,▁최고,0.581895


# 새로운 데이터에 적용

새로운 데이터

In [59]:
new_data = ['뽀로로는 정말 재미있는 영화다.', '이런 영화를 만들다니 감독은 무슨 생각이냐?']

토큰화 데이터 불러오기

In [62]:
from sentencepiece import SentencePieceProcessor
sp = SentencePieceProcessor()
sp.Load("nsmc.model")

True

`cv`를 저장할 때 `.tokenizer`를 비우고 저장하지 않았으므로 다시 설정해준다.

In [63]:
cv.tokenizer = sp.encode_as_pieces

새로운 데이터를 TDM으로 변환한다. 이미 TDM의 형식이 정해져있으므로 `fit_transform` 대신 `transform`을 사용한다.

In [64]:
x_new = cv.transform(new_data)

새로운 데이터에서 각 리뷰가 긍정(`1`)일 확률을 예측한다.

In [65]:
model.predict(x_new.toarray())

array([[0.7371192 ],
       [0.33626053]], dtype=float32)

In [66]:
model.predict_classes(x_new.toarray())

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[1],
       [0]], dtype=int32)